 ```{epigraph}
Programming starts with types and functions. You probably have some preconceptions
about what types and functions are: get rid of them! They will cloud your mind. 
 
 --Bartosz Milewski
 ```

## Localization

In [110]:
type Blocks a = (Fractional a) => [[a]]

In [74]:
:k Blocks

Blocks :: * -> *

### blocks
`blocks` is a functor level function that maps a given functor `[a]` to functor `[[a]]`, so the definition is `blocks::[a]->[[a]]`.

In [111]:
blocks::(Fractional a)=>Int->[a]->Blocks a
blocks n x
    | n<=0 = [x]
blocks _ [] = []
blocks n xs = take n xs:blocks n (drop n xs)

In [112]:
blocks 2 [0..7]

[[0.0,1.0],[2.0,3.0],[4.0,5.0],[6.0,7.0]]

In [113]:
take 5 $ blocks 2 [0..]

[[0.0,1.0],[2.0,3.0],[4.0,5.0],[6.0,7.0],[8.0,9.0]]

### oblocks

In [105]:
data TripleBlocks a = (Fractional a)=>Triple{left::[a],center::[a],right::[a]}
type OBlocks a = (Fractional a)=>[TripleBlocks a]

In [132]:
Triple{left=[1.0,2.0],center=[3.0,4.0],right=[5.0,6.0]}

: 

In [116]:
:k TripleBlocks

TripleBlocks :: * -> *

#### Sliding windows --- windowed::Int->[a]->[[a]]

In [15]:
-- windowed on list, returning windows with n elements  each
windowed :: Int->[a]->[[a]]
windowed n x
    | n<=0 = [x]
windowed _ [] = []
windowed n (x:xs)
    | length(take n (x:xs))<n = [] -- `take n` makes sure it works on infinite list
    | otherwise = w:windowed n xs
    where w=take n (x:xs)
-- without guarantee each window has n element, then the following works:
-- windowed n x = Data.List.transpose $ take n $ tails x
-- or
-- windowed n x = map (tak n) $ tails x



In [16]:
take 5 $ windowed 3 [0..]

[[0,1,2],[1,2,3],[2,3,4],[3,4,5],[4,5,6]]

In [17]:
windowed 3 [0..7]

[[0,1,2],[1,2,3],[2,3,4],[3,4,5],[4,5,6],[5,6,7]]

### Padding

In [128]:
class Padding a where
    pad::(Fractional a)=>Int->[a]->OBlocks a

data ZeroPadding

In [129]:
instance Padding ZeroPadding where
    pad n a = Triple{left=replicate n 0.0, center=a, right=replicate n 0.0}

: 

In [124]:
:t repeat

repeat :: forall a. a -> [a]

## Decimation

In [18]:
decimation::Int->[a]->[a]
decimation n x
    | n<2 = x
decimation _ [] = []
decimation n (x:xs) = x:decimation n (drop (n-1) xs)

In [19]:
decimation 2 [1,2,3]

[1,3]

In [20]:
take 5 $ decimation 2 [0..]

[0,2,4,6,8]

In [21]:
decimation 1 [1..5]

[1,2,3,4,5]

## Translation

## Utils
### Linespace

In [22]:
linespace' :: (Fractional a, Ord a)=>a->a->Int->[a]
linespace' _ _ n
    | n<=1 = []
linespace' a1 a2 _
    | a2<=a1 = []
linespace' a1 a2 n = map (\ i -> a1+dt*fromIntegral i) [0..(n-1)]
    where
        dt=(a2-a1)/fromIntegral(n-1)
--linespace a1 a2 n = [let dt=(a2-a1)/(n-1) in a1+i*dt | i<-[0..(n-1)]]

In [23]:
linespace :: (Fractional a, Ord a)=>a->a->Int->[a]
linespace a1 a2 _
    | a2<=a1 = []
linespace a b n
    | n<=0 = []
    | n==1 = [a, b]
linespace a1 a2 n = [let dt=(a2-a1)/fromIntegral(n-1) in a1+dt*fromIntegral i | i<-[0..(n-1)]]

In [24]:
linespace' (-1) 1.0 8

[-1.0,-0.7142857142857143,-0.4285714285714286,-0.1428571428571429,0.1428571428571428,0.4285714285714284,0.7142857142857142,1.0]

In [25]:
linespace 1.0 (-1) 1

[]

In [26]:
len::[a]->Int
len [] = 0
len xs = foldr (\ x -> (+) 1) 0 xs

In [27]:
len [0..7]

8

In [28]:
foo=len [0..]

### Binary Tree

In [30]:
data Tree a = EmptyTree | Node a (Tree a) (Tree a) deriving (Show, Read, Eq) 

## Visualization

In [29]:
import Graphics.Rendering.Chart.Backend.Diagrams

: 